In [ ]:
pip install split-folders


In [ ]:

import splitfolders

# dataset path
input_folder = '/content/drive/MyDrive/SPAM IMAGE dataset/'

# Split with a ratio. To only split into training & validation set, set a tuple 'ratio', i.e,
# Train(70%), Val(20), Test(10)
splitfolders.ratio(input_folder, output='/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2',
                   seed=42, ratio=(.7,.2,.1),
                   group_prefix=None)

In [1]:
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf


In [3]:


def create_model(base_model, img_size=(224, 224), num_classes=1):
    # Add new classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)

    # Combine base model with new classification layers
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze layers of the pre-trained model
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Define a function to train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )
    return history

# Function to load a saved model
def load_saved_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

# Example usage:

# Define your dataset directory paths
train_dir = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2/train'
validation_dir = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2/val'

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 1225 images belonging to 2 classes.
Found 349 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3, MobileNetV2, DenseNet121

# Choose the base model
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_densenet50 = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create models using the function for different base models
model_vgg16 = create_model(base_model_vgg16)
model_vgg19 = create_model(base_model_vgg19)
model_resnet50 = create_model(base_model_resnet50)
model_inceptionv3 = create_model(base_model_inceptionv3)
model_mobilenetv2 = create_model(base_model_mobilenetv2)
model_densenet50 = create_model(base_model_densenet50)

# Train the models
history_vgg16 = train_model(model_vgg16, train_generator, validation_generator)
history_vgg19 = train_model(model_vgg19, train_generator, validation_generator)
history_resnet50 = train_model(model_resnet50, train_generator, validation_generator)
history_inceptionv3 = train_model(model_inceptionv3, train_generator, validation_generator)
history_mobilenetv2 = train_model(model_mobilenetv2, train_generator, validation_generator)
history_densenet50 = train_model(model_densenet50, train_generator, validation_generator)

# Save the models
model_vgg16.save('spam_image_filter_model_vgg16.h5')
model_vgg19.save('spam_image_filter_model_vgg19.h5')
model_resnet50.save('spam_image_filter_model_resnet50.h5')
model_inceptionv3.save('spam_image_filter_model_inceptionv3.h5')
model_mobilenetv2.save('spam_image_filter_model_mobilenetv2.h5')
model_densenet50.save('spam_image_filter_model_densenet50.h5')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('spam_image_filter_model.h5')



# Function to preprocess uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to predict whether the image contains spam or not

def predict_spam(image_path):
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0][0]  # Return the prediction value (probability)

# Provide the path to the uploaded image for prediction
uploaded_image_path = '/content/drive/MyDrive/SPAM IMAGE dataset/SpamImages/05D89BDjGS.jpg'

# Predict whether the uploaded image contains spam
prediction_result = predict_spam(uploaded_image_path)

# Threshold for considering if it's spam or not (adjust as needed)
spam_threshold = 0.5

if prediction_result >= spam_threshold:
    print("The uploaded image is predicted as SPAM.")
else:
    print("The uploaded image is predicted as NOT SPAM.")



# predict individual data using all models

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# import numpy as np

# # Load the trained model based on the model name
# def load_saved_model(model_name):
#     model_path = f'spam_image_filter_model_{model_name}.h5'
#     return tf.keras.models.load_model(model_path)

# # Function to preprocess uploaded image
# def preprocess_image(image_path):
#     img = image.load_img(image_path, target_size=(224, 224))
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     return img_array

# # Function to predict whether the image contains spam or not
# def predict_spam(model, image_path):
#     processed_img = preprocess_image(image_path)
#     prediction = model.predict(processed_img)
#     return prediction[0][0]  # Return the prediction value (probability)

# # Provide the path to the uploaded image for prediction
# uploaded_image_path = '/content/drive/MyDrive/SPAM IMAGE dataset/SpamImages/05D89BDjGS.jpg'

# # List of model names
# model_names = ['vgg16', 'vgg19', 'resnet50', 'inceptionv3', 'mobilenetv2', 'densenet50']

# # Iterate through models and perform predictions
# for model_name in model_names:
#     # Load the model
#     model = load_saved_model(model_name)

#     # Predict whether the uploaded image contains spam
#     prediction_result = predict_spam(model, uploaded_image_path)

#     # Threshold for considering if it's spam or not (adjust as needed)
#     spam_threshold = 0.5

#     # Print prediction result for each model
#     if prediction_result >= spam_threshold:
#         print(f"The uploaded image using {model_name.upper()} model is predicted as SPAM.")
#     else:
#         print(f"The uploaded image using {model_name.upper()} model is predicted as NOT SPAM.")
